In [0]:
import json

X = json.load(open('RNN_train_x.json'))
Y = json.load(open('RNN_train_y.json'))

In [0]:
from tqdm.notebook import tqdm
import torch

class translator :
    def __init__(self, words, min_count, empty=False) :
      if not empty :
        npwords = np.array(words)
        count = [(word,(npwords==word).sum()) for word in tqdm(np.unique(words))]
        count = sorted(count, key=lambda x: x[1], reverse=True)
        d = -1
        while True :
          d += 1
          if count[d][1] < min_count :
            break
        self.dim = d+1
        self.mapping = {count[i][0] : i for i in range(d)}

    def one_hot(self, i) :
      a = torch.zeros(self.dim)
      a[i] = 1
      return a

    def __getitem__(self, i, tensor=True) :
        if type(i)==type('') :
          if tensor :
            return self.one_hot(self.mapping.get(i, self.dim-1))
          else :
            return self.one_hot(self.mapping.get(i, self.dim-1)).tolist()
        
        elif type(i)==type(1) :
          if tensor :
            return self.one_hot(i)
          else :
            return self.one_hot(i).tolist()
        
        elif type(i)==type(list()) :
          if tensor :
            return torch.tensor(list(map(lambda x: self.__getitem__(x, False), i)))
          else :
            return list(map(lambda x: self.__getitem__(x, False), i))

        else :
            return self.__getitem__(i.tolist())

    def get_item(self, i) :
        if type(i)==type('') :
            return self.mapping.get(i, self.dim-1)
        if type(i)==type(1) :
            return i
        elif type(i)==type(list()) :
            return torch.tensor(list(map(self.get_item, i)))
        else :
            return self.get_item(i.tolist())

    def save_to_file(self, name) :
      json.dump({'dim': self.dim, 'mapping': self.mapping}, open(name,'w'))

    @classmethod
    def load_from_file(cls, name) :
      f = json.load(open(name))
      self = cls([],0,True)
      self.dim = f['dim']
      self.mapping = f['mapping']
      return self

onehoter = translator.load_from_file('RNN_one_hoter.json')

In [0]:
import pandas as pd

pos_indexer = json.load(open('pos_indexer.json'))
def pos_index(l) :
  return list(map(pos_indexer.__getitem__, l))
pos_names = pd.read_csv('pos_list.csv')
def pos_name(l) :
  return list(map(lambda x: pos_names.iloc[int(x)][0], l))

In [0]:
import torch.nn as nn

class MyRNN(nn.Module):
    def __init__(self, n_inputs, n_neurons, n_classes):
        super(MyRNN, self).__init__()
        
        self.rnn = nn.RNNCell(n_inputs, n_neurons)
        self.l = nn.Linear(n_neurons, n_classes)
        self.hx = torch.randn(1, n_neurons)
        
    def forward(self, X):
        output = []

        # for each time step
        for i in range(len(X)):
            self.hx = self.rnn(X[i].reshape(1,-1), self.hx)
            o = torch.sigmoid(self.l(self.hx))
            output.append(o)
        
        return output, self.hx

In [0]:
from tqdm.notebook import trange
import numpy as np
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

def usable_data(X,Y) :
  x = onehoter[X]
  y = torch.tensor(pos_index(Y))
  return x,y

def train_step(model, X, Y, lr) :
  optimizer = optim.SGD(model.parameters(), lr=lr)
  X,Y = usable_data(X,Y)
  size = len(Y)
  optimizer.zero_grad()
  outputs, _ = model(X)
  loss = list(map(lambda i: criterion(outputs[i], torch.tensor([Y[i]])), range(size)))
  loss = sum(loss) / size
  loss.backward()
  optimizer.step()
  model.hx = model.hx.detach()
  return loss

def train(model, X, Y, lr, epoch) :
  length = len(Y)
  index = np.arange(length)
  losses = list()
  for e in tqdm(range(epoch)) :
      np.random.shuffle(index)
      ls = list()
      for i in index :
        l = train_step(model, X[i], Y[i], lr)
        l = round(float(l.data),3)
        ls.append(l)
      losses.append(sum(ls)/len(ls))
  return losses

In [0]:
model = MyRNN(onehoter.dim, 60, len(pos_indexer))
losses = train(model,X,Y,0.01,20)

In [0]:
import json
json.dump(losses,open('losses.json','w'))
from google.colab import files
files.download('losses.json') 

KeyboardInterrupt: ignored

In [0]:
import matplotlib.pyplot as plt
plt.plot(losses)
plt.titile('RNN trainong')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.show

In [0]:
losses += train(model,X,Y,0.001,10)
json.dump(losses,open('losses.json','w'))
files.download('losses.json') 

In [0]:
import matplotlib.pyplot as plt
plt.plot(losses)
plt.titile('RNN trainong')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.show

In [0]:
# def test_step(model, X, Y) :
#   X,Y = usable_data(X,Y)
#   size = len(Y)
#   with torch.no_grad():
#       outputs,_ = model(X)
#       _, predicted = torch.max(outputs.data, 1)
      
#   model.hx = model.hx.detach()
#   return loss

# def Test(NET, testloader) :
#   correct = 0
#   total = 0
#   with torch.no_grad():
#       for data in testloader:
#           images, labels = data
#           images, labels = images.to(device), labels.to(device)
#           outputs = NET(images)
#           _, predicted = torch.max(outputs.data, 1)
#           total += labels.size(0)
#           correct += (predicted == labels).sum().item()

#   print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

# def train_step(model, X, Y, lr) :
#   optimizer = optim.SGD(model.parameters(), lr=lr)
#   X,Y = usable_data(X,Y)
#   X,Y = X.to(device), Y.to(device)
#   size = len(Y)
#   optimizer.zero_grad()
#   outputs, _ = model(X)
#   loss = list(map(lambda i: criterion(outputs[i], torch.tensor([Y[i]])), range(size)))
#   loss = sum(loss) / size
#   loss.backward()
#   optimizer.step()
#   model.hx = model.hx.detach()
#   return loss

# def train(model, X, Y, lr, epoch) :
#   length = len(Y)
#   index = np.arange(length)
#   losses = list()
#   for e in tqdm(range(epoch)) :
#       np.random.shuffle(index)
#       ls = list()
#       for i in tqdm(index) :
#         l = train_step(model, X[i], Y[i], lr)
#         l = round(float(l.data),3)
#         ls.append(l)
#       losses.append(sum(ls)/len(ls))
#   return losses